In [1]:
import numpy as np
import pickle
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib
from matplotlib import pyplot as plt
from sklearn.metrics import f1_score, accuracy_score
from tensorflow.keras import models
from tensorflow.keras.layers import Dense, Input, Dropout, LSTM, Masking, Bidirectional, Concatenate, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

In [2]:
def plotTSNE(tsne_results, Y_test):
    plt.figure(figsize=(12,12))
    label = Y_test
    colors = ['blue', 'red']
    plt.scatter(x=tsne_results[:,0], y=tsne_results[:,1], c=label, cmap=matplotlib.colors.ListedColormap(colors), s=3)

In [3]:
X_pca_mean = np.asarray(pd.read_csv("data/diabetes/X_pca_mean.csv", header=None))[:6000]
X_val_pca_mean = np.asarray(pd.read_csv("data/diabetes/X_pca_mean.csv", header=None))[6000:8000]
X_test_pca_mean = np.asarray(pd.read_csv("data/diabetes/X_pca_mean.csv", header=None))[8000:]

X_lda_mean = np.asarray(pd.read_csv("data/diabetes/X_lda_mean.csv", header=None))[:6000]
X_val_lda_mean = np.asarray(pd.read_csv("data/diabetes/X_lda_mean.csv", header=None))[6000:8000]
X_test_lda_mean = np.asarray(pd.read_csv("data/diabetes/X_lda_mean.csv", header=None))[8000:]

X_pca_max = np.asarray(pd.read_csv("data/diabetes/X_pca_max.csv", header=None))[:6000]
X_val_pca_max = np.asarray(pd.read_csv("data/diabetes/X_pca_max.csv", header=None))[6000:8000]
X_test_pca_max = np.asarray(pd.read_csv("data/diabetes/X_pca_max.csv", header=None))[8000:]

X_lda_max = np.asarray(pd.read_csv("data/diabetes/X_lda_max.csv", header=None))[:6000]
X_val_lda_max = np.asarray(pd.read_csv("data/diabetes/X_lda_max.csv", header=None))[6000:8000]
X_test_lda_max = np.asarray(pd.read_csv("data/diabetes/X_lda_max.csv", header=None))[8000:]

X_pca_conc = np.asarray(pd.read_csv("data/diabetes/X_pca_conc.csv", header=None))[:6000]
X_val_pca_conc = np.asarray(pd.read_csv("data/diabetes/X_pca_conc.csv", header=None))[6000:8000]
X_test_pca_conc = np.asarray(pd.read_csv("data/diabetes/X_pca_conc.csv", header=None))[8000:]

X_lda_conc = np.asarray(pd.read_csv("data/diabetes/X_lda_conc.csv", header=None))[:6000]
X_val_lda_conc = np.asarray(pd.read_csv("data/diabetes/X_lda_conc.csv", header=None))[6000:8000]
X_test_lda_conc = np.asarray(pd.read_csv("data/diabetes/X_lda_conc.csv", header=None))[8000:]

Y = np.asarray(pd.read_csv("data/diabetes/Y.csv", header=None)).reshape(-1).astype(int)
Y_val = np.asarray(pd.read_csv("data/diabetes/Y_val.csv", header=None)).reshape(-1).astype(int)
Y_test = np.asarray(pd.read_csv("data/diabetes/Y_test.csv", header=None)).reshape(-1).astype(int)
header = pd.read_csv("data/diabetes/diab_train.csv").keys()

In [111]:
X = X_pca_max
X_val = X_val_pca_max
X_test = X_test_pca_max

In [112]:
X2 = np.vstack([X, X_val])
Y2 = np.hstack([Y, Y_val])

In [5]:
print(header)

Index(['Unnamed: 0', 'race', 'gender', 'age', 'weight', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'payer_code', 'medical_specialty', 'num_lab_procedures',
       'num_procedures', 'num_medications', 'number_outpatient',
       'number_emergency', 'number_inpatient', 'diag_1', 'diag_2', 'diag_3',
       'number_diagnoses', 'max_glu_serum', 'A1Cresult', 'metformin',
       'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride',
       'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide.metformin', 'glipizide.metformin',
       'glimepiride.pioglitazone', 'metformin.rosiglitazone',
       'metformin.pioglitazone', 'change', 'diabetesMed', 'readmitted',
       'diag_1_desc', 'diag_2_desc', 'diag_3_desc'],
      dtype='object')


In [171]:
def model():
    x = Input(shape=(65,))
    x_d = Input(shape=(10, 3174))
    x_dis = Input(shape=(4, 44))
    
    rnn_d = LSTM(30)(x_d)
    rnn_dis = LSTM(20)(x_dis)
    
    conc = Concatenate()([x, rnn_d, rnn_dis])

    dense = Dense(400, activation='relu', name="dense")(x)
    dense = Dropout(.1)(dense)
    dense = BatchNormalization()(dense)
    dense = Dense(300, activation='relu', name="dense2")(dense)
    dense = Dropout(.1)(dense)
    dense = BatchNormalization()(dense)
    dense = Dense(200, activation='relu', name="dense3")(dense)
    dense = Dropout(.1)(dense)
    dense = BatchNormalization()(dense)
    dense = Dense(100, activation='relu', name="dense4")(dense)
    dense = Dropout(.1)(dense)
    dense = BatchNormalization()(dense)
    y = Dense(1, activation='sigmoid', name="y")(dense)

    model = models.Model([x, x_d, x_dis], y, name="model")
    model.compile(optimizer='Adam', loss="binary_crossentropy", metrics=['acc'])

    return model

In [172]:
m = model()

In [173]:
m.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, 65)]         0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 400)          26400       input_12[0][0]                   
__________________________________________________________________________________________________
dropout_12 (Dropout)            (None, 400)          0           dense[0][0]                      
__________________________________________________________________________________________________
batch_normalization_12 (BatchNo (None, 400)          1600        dropout_12[0][0]                 
______________________________________________________________________________________________

In [174]:
Y.shape

(6000,)

In [175]:
cbs = []
cbs.append(EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1))
cbs.append(ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2))        
m.fit([X, X_diagnoses, X_discharge_disposition], Y, shuffle=True, epochs=100, \
      validation_data=([X_val], Y_val), \
      callbacks=cbs)

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 3 array(s), but instead got the following list of 1 arrays: [array([[0.03523807, 0.        , 0.08222217, ..., 0.        , 0.        ,
        0.        ],
       [0.03279521, 0.        , 0.05465868, ..., 0.        , 0.        ,
        0.        ],
       [0.0...

In [16]:
f1_score(Y_val, m.predict(X_val).round().astype(int))

0.4016326530612245

In [45]:
print(m.predict(X_val).round().astype(int).reshape(-1)[:30])
print(Y_val.astype(int).reshape(-1)[:30])

[0 0 1 0 1 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 1 1 0 0 0 1]
[0 1 1 1 0 1 0 1 0 0 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1]


In [115]:
import lightgbm as lgbm
param_grid = {
    'boosting_type': ['gbdt', 'dart'],
    'num_leaves': [15, 20, 23, 25, 30, 40, 80],
    'reg_alpha': [0.5, 0.7, 0.9, 1.1, 1.3, 1.6],
    'min_data_in_leaf': [15, 19, 22, 26, 30, 35, 45, 80],
    'max_bin': [150, 200, 250, 300, 350, 400, 450],
    'feature_fraction': [.2, .3, .4, .6, .8]
    }

lgb_estimator = lgbm.LGBMClassifier(objective='binary', 
                                    class_weight='balanced',
                                    eval_metric='f1',
                                    jobs=-1)
gkf = KFold(n_splits=5, shuffle=True, random_state=42).split(X=X2, y=Y2)
gsearch = GridSearchCV(estimator=lgb_estimator, param_grid=param_grid, cv=gkf, verbose=3, n_jobs=-1, scoring='f1')

In [ ]:
lgb_model = gsearch.fit(X=X2, y=Y2)
print(lgb_model.best_params_, lgb_model.best_score_)

Fitting 5 folds for each of 23520 candidates, totalling 117600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:   15.4s
[Parallel(n_jobs=-1)]: Done 496 tasks      | elapsed:   27.6s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   42.6s
[Parallel(n_jobs=-1)]: Done 1136 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 1552 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 2032 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 2576 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 3184 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 3856 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 4592 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 5392 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done 6256 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done 7184 tasks      | e

In [64]:
lgb_estimator = lgbm.LGBMClassifier(boosting_type='gbdt', 
                                    min_data_in_leaf=30, \
                                    num_leaves=21, 
                                    max_bin=280, 
                                    reg_alpha=1.3,
                                    objective='binary', \
                                    class_weight='balanced',
                                    feature_fraction=.26,
                                    num_boost_round=1000
                                   )
lgb_estimator.fit(np.vstack([X, X_val]),
                  np.hstack([Y, Y_val]),
                  eval_set=(X_val, Y_val), 
                  early_stopping_rounds=5, 
                  eval_metric='f1', verbose=0, 
                  callbacks = [lgbm.reset_parameter(learning_rate = np.linspace(0.12, 0.05, 20).tolist()+[0.05]*980)])

/home/vescovo/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


LGBMClassifier(boosting_type='gbdt', class_weight='balanced',
               colsample_bytree=1.0, feature_fraction=0.26,
               importance_type='split', learning_rate=0.1, max_bin=280,
               max_depth=-1, min_child_samples=20, min_child_weight=0.001,
               min_data_in_leaf=30, min_split_gain=0.0, n_estimators=100,
               n_jobs=-1, num_boost_round=1000, num_leaves=21,
               objective='binary', random_state=None, reg_alpha=1.3,
               reg_lambda=0.0, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

In [65]:
print(f1_score(Y_test, lgb_estimator.predict(X_test).round().astype(int)))
print(lgb_estimator.predict(X_test)[:30])
print((np.round(np.max(lgb_estimator.predict_proba(X_test)[:30], axis=1), 1)*10).astype(int))
print(Y_test[:30])

0.5446371226718048
[1 0 0 1 0 0 1 0 1 0 0 1 1 0 0 1 0 0 0 1 0 0 0 1 1 0 0 0 1 1]
[6 7 8 7 8 7 5 8 5 5 8 6 5 9 7 5 7 8 7 6 6 6 6 7 7 8 5 5 9 6]
[0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 1 0 1 1 0 1 1 0]


In [100]:
np.asarray(list(zip(np.sort(lgb_estimator.feature_importances_), np.argsort(lgb_estimator.feature_importances_))))[-60:]

array([[ 4, 90],
       [ 5, 60],
       [ 5, 78],
       [ 5, 48],
       [ 5, 21],
       [ 5, 94],
       [ 6, 66],
       [ 6, 72],
       [ 7, 88],
       [ 7, 92],
       [ 8, 86],
       [ 8, 76],
       [ 8, 22],
       [ 9, 68],
       [ 9, 91],
       [10, 81],
       [10, 75],
       [10, 45],
       [11, 80],
       [11, 87],
       [14, 65],
       [15, 89],
       [15, 93],
       [15,  7],
       [15, 69],
       [17, 96],
       [17, 24],
       [17, 85],
       [17, 13],
       [18, 79],
       [18, 44],
       [18, 82],
       [18, 73],
       [20, 14],
       [20, 38],
       [22, 74],
       [22, 39],
       [22,  1],
       [23,  3],
       [24, 84],
       [25, 10],
       [26, 70],
       [26, 62],
       [26,  5],
       [29, 83],
       [33, 32],
       [33, 71],
       [35, 77],
       [36,  6],
       [36,  9],
       [38,  2],
       [39, 54],
       [39, 42],
       [42, 97],
       [46,  4],
       [46, 95],
       [52,  0],
       [65, 12],
       [66,  8

In [67]:
from sklearn.svm import SVC

In [70]:
param_grid = {
    'C': [30, 35, 40, 45, 50],
    'kernel': ['linear', 'rbf'],
    #'degree': [2, 3, 4, 5],
    'gamma': ['auto'],
    'shrinking': [True],
    'decision_function_shape': ['ovo', 'ovr']
    }

clf = SVC(class_weight='balanced')

gkf = KFold(n_splits=5, shuffle=True, random_state=42).split(X=X2, y=Y2)
gsearch_clf = GridSearchCV(estimator=clf, param_grid=param_grid, cv=gkf, verbose=3, n_jobs=-1, scoring='f1')

In [71]:
clf_model = gsearch_clf.fit(X=X2, y=Y2)
print(clf_model.best_params_, clf_model.best_score_)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   30.7s
/home/vescovo/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  2.7min finished


{'C': 50, 'decision_function_shape': 'ovo', 'gamma': 'auto', 'kernel': 'linear', 'shrinking': True} 0.5315334168897159


In [85]:
clf = SVC(class_weight='balanced', C=100, kernel='linear', degree=2, gamma='auto', shrinking=True, decision_function_shape='ovo')
clf.fit(X2, Y2)

SVC(C=100, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovo', degree=2, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [84]:
f1_score(Y_test, clf.predict(X_test).round().astype(int))

0.46586910626319494

In [87]:
from sklearn.ensemble import AdaBoostClassifier

In [89]:
param_grid = {
    'C': [0.1, 0.5, 1, 2],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'degree': [2, 3, 4, 5],
    'gamma': ['auto', 'scale'],
    'shrinking': [True, False],
    'decision_function_shape': ['ovo', 'ovr']
    }

ada = AdaBoostClassifier(n_estimators=1000, random_state=0)

gkf = KFold(n_splits=5, shuffle=True, random_state=42).split(X=X2, y=Y2)
gsearch_ada = GridSearchCV(estimator=ada, param_grid=param_grid, cv=gkf, verbose=3, n_jobs=-1, scoring='f1')

In [90]:
ada_model = gsearch_ada.fit(X=X2, y=Y2)
print(ada_model.best_params_, ada_model.best_score_)

Fitting 5 folds for each of 512 candidates, totalling 2560 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


ValueError: Invalid parameter C for estimator AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=1000, random_state=0). Check the list of available parameters with `estimator.get_params().keys()`.

In [ ]:
ada = AdaBoostClassifier(n_estimators=1000, random_state=0)
ada.fit(X, Y)

In [36]:
f1_score(Y_val, ada.predict(X_val))

0.4201930215293245

In [76]:
from sklearn.neighbors import KNeighborsClassifier

In [100]:
param_grid = {
    'n_neighbors': [3, 4, 5, 6, 10, 15],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
    }

knn = KNeighborsClassifier(4, weights='distance')

gkf = KFold(n_splits=5, shuffle=True, random_state=42).split(X=X2, y=Y2)
gsearch_knn = GridSearchCV(estimator=knn, param_grid=param_grid, cv=gkf, verbose=3, n_jobs=-1, scoring='f1')

In [101]:
knn_model = gsearch_knn.fit(X=X2, y=Y2)
print(knn_model.best_params_, knn_model.best_score_)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:   18.0s


{'algorithm': 'auto', 'n_neighbors': 3, 'weights': 'distance'} 0.4299845194601132


[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:   29.2s finished


In [102]:
knn = KNeighborsClassifier(3, weights='distance')
knn.fit(X2, Y2)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='distance')

In [103]:
f1_score(Y_test, knn.predict(X_test))

0.4078431372549019

In [104]:
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

In [105]:
kernel = 1.0 * RBF(1.0)

In [106]:
gpc = GaussianProcessClassifier(kernel=kernel, random_state=0, n_jobs=-1).fit(X2, Y2)

In [107]:
f1_score(Y_test, gpc.predict(X_test))

0.39530988274706874

In [108]:
from sklearn import tree

In [109]:
dt = tree.DecisionTreeClassifier(n_jobs=-1)
dt = dt.fit(X2, Y2)

In [110]:
f1_score(Y_test, dt.predict(X_test))

0.4542079207920792